In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
df.head()

In [17]:
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
import matplotlib.pyplot as plt

In [4]:
df['diagnosis'].value_counts()

In [5]:
df.isnull().sum()

In [6]:
df.drop('Unnamed: 32',axis=1,inplace=True)
df.head()

In [7]:
df.drop('id',axis=1,inplace=True)
df.head()

In [8]:
x=df.drop('diagnosis',axis=1)

In [9]:
x.head()

In [10]:
df.loc[(df.diagnosis == 'B'),'diagnosis']='0'
df.loc[(df.diagnosis == 'M'),'diagnosis']='1'
df.head(10)

In [11]:
df['diagnosis'] = df['diagnosis'].apply(pd.to_numeric, errors='coerce')
y=df['diagnosis']
y.head()

In [12]:
x = np.asarray(x)
y=np.array(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [13]:
from sklearn.model_selection import KFold

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)

In [18]:
import tensorflow
learningrate=0.1
from keras import optimizers
neurons=3
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier = Sequential()
    classifier.add(Dense(units=neurons,kernel_initializer='he_uniform',activation='relu',input_dim=30))
    classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    sgd = tensorflow.keras.optimizers.SGD(learning_rate=learningrate)
    classifier.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
    model = classifier.fit(X_train,y_train,batch_size=100,epochs=20)
    y_pred = classifier.predict(X_test)
    y_pred = (y_pred>0.5)
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_pred))
    from sklearn.metrics import roc_curve, auc
    nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test, y_pred)
    auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
    plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)
    plt.show()
    learningrate+=0.1
    learningrate=learningrate/1.0
    neurons+=1
    if(neurons==6):
        neurons=3

In [19]:
classifier = Sequential()

In [20]:
#first hidden layer
classifier.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu',input_dim=30))

In [21]:
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [22]:
classifier.summary()

In [23]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model = classifier.fit(X_train,y_train,batch_size=100,epochs=100)

In [24]:
y_pred = classifier.predict(X_test)

In [25]:
y_pred = (y_pred>0.5)

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [29]:
from sklearn.metrics import roc_curve, auc
nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.title("ROC Curve")
plt.ylabel('true possitive rate')
plt.xlabel('false possitive rate')
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)

In [31]:

plt.plot(model.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [32]:
plt.plot(model.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()